# Square ice shelf tutorial

This is an example of velocity computation in steady state for a square ice shelf. We follow the tutorial as in the ice-sheet and Sea-level System Model (ISSM): [https://issm.jpl.nasa.gov/documentation/tutorials/squareiceshelf/](https://issm.jpl.nasa.gov/documentation/tutorials/squareiceshelf/), which is the C++ version of ice sheet model.


## Install the package

To use this tutorial one need to install `dJUICE` using the following commands:

```julia
]add https://github.com/DJ4Earth/dJUICE.jl
```

After installing, import the package:

In [ ]:
using dJUICE

## Initialize the model


All the data belonging to a model (geometry, node coordinates, results, etc.) is held in the same object `Model`. To create a new model, one can use the following command:

In [ ]:
md = model();

This will create a new model named `md` whose `struct` is `Model`. The information contained in the model `md` are grouped by `struct`, that contain fields related to a particular aspect of the model (e.g. mesh, ice geometry, material properties, friction, stressbalance solution, results of the runs, etc.). When one creates a new model, all these fields are empty, but `md` is ready to be used as a model. The list of these classes is displayed by running:

In [ ]:
md

## Generate Mesh

Use `md=triangle(md,domainname,resolution)` to generate an unstructured triangular mesh based on the domain outline defined in the file `domainname`, with a characteristic length `resolution` for the mesh.

We start with a square domain of $[0, 10^6]\times[0,10^6]$ with $5\times10^3~$ mesh resolution, all the units are in meters. Indeed, in `dJUICE` we use [SI](https://en.wikipedia.org/wiki/International_System_of_Units) unit system.

In [ ]:
md = triangle(md,issmdir()*"/test/Exp/Square.exp",50000.) 

# may need to install GLMakie
plotmodel(md, "mesh")

Now, let's use `IJulia.load(issmdir()*"/test/Exp/Square.exp")` or `readlines(issmdir()*"/test/Exp/Square.exp")` to see the `Square.exp` file:

In [ ]:
readlines(issmdir()*"/test/Exp/Square.exp")

## Set the geometry

We define the ice geometry on the given domain. 
### Ice thickness
Let's set the `thickness` of the ice shelf to be 
$$H(x,y)=h_{\max} + \frac{h_{\min}-h_{\max}}{L_y}y+0.1\frac{h_{\min}-h_{\max}}{L_x}x$$
where $h_\min=300$ and $h_\max=1000$

In [ ]:
hmin=300.
hmax=1000.
ymin=minimum(md.mesh.y)
ymax=maximum(md.mesh.y)
xmin=minimum(md.mesh.x)
xmax=maximum(md.mesh.x)

md.geometry.thickness = hmax .+ (hmin-hmax)*(md.mesh.y .- ymin)./(ymax-ymin) .+ 
                            0.1*(hmin-hmax)*(md.mesh.x .- xmin)./(xmax-xmin)
# plot the ice thickness
plotmodel(md, md.geometry.thickness)

### Ice base
Because the ice shelf is floating, we can determine the `base` of the ice by the flotation criteria
$$b=-\frac{\rho_i}{\rho_w}*H,$$ where $\rho_i$ and $\rho_w$ are the density of ice and water. The physical constants are given by default in `md.materials` when initialize the model. 

In [ ]:
md.geometry.base = -md.materials.rho_ice/md.materials.rho_water*md.geometry.thickness
plotmodel(md, md.geometry.base)

### Ice surface

The `surface` of the ice is then calculated as $s=b+H$. 

In [ ]:
md.geometry.surface   = md.geometry.base+md.geometry.thickness
plotmodel(md, md.geometry.surface)

### Bedrock
In this tutorial, we are going to work on an ice shelf. To gurantee the ice is completely floating, we set the sea `bed` (bedrock) elevation to be 10 meters deeper than the `base` of the ice:

In [ ]:
md.geometry.bed       = md.geometry.base .-10
plotmodel(md, md.geometry.bed)

## Set intitial conditions

We set the whole domain to be ice covered, as an ice shelf.
The initial velocity is set to zeros. 

In [ ]:
# set ice mask 
md = setmask(md,"all","")
md.initialization.vx = zeros(md.mesh.numberofvertices)
md.initialization.vy = zeros(md.mesh.numberofvertices);

## Set physical parameters

Ice in large scale is a non-Neutownian, viscos fluid. Its governing dynamics are described by the Shelfy Stream Approximation (SSA), expressed as a system of PDEs: 
\begin{equation}\label{eq:PDEs}
    \nabla\cdot\boldsymbol{\sigma}+{\boldsymbol{\tau}}_b=\rho_i g H \nabla s
\end{equation}
where $\boldsymbol{\tau}_b=(\tau_{bx}, \tau_{by})^T$ represents the basal shear stress, $\rho_i$ is the ice density, $g$ is the gravitational acceleration. $\boldsymbol{\sigma}$ is the stress tensor of the SSA model defined as
\begin{equation}\label{eq:stresstensor}
    \boldsymbol{\sigma} = \mu H
    \begin{bmatrix}
         \displaystyle 4\frac{\partial u}{\partial x}+2\frac{\partial v}{\partial y} 
         & 
         \displaystyle \frac{\partial u}{\partial y}+\frac{\partial v}{\partial x}  \\
         \\
         \displaystyle \frac{\partial u}{\partial y}+\frac{\partial v}{\partial x} 
         &
         \displaystyle 2\frac{\partial u}{\partial x}+4\frac{\partial v}{\partial y}
    \end{bmatrix}.
\end{equation}
The ice viscosity, $\mu$, is determined by Glen's flow-law, which in two dimensions reads: 
\begin{equation}
    \mu =\frac{B}{2}\left( \left(\frac{\partial u}{\partial x}\right)^2 + \left(\frac{\partial v}{\partial y}\right)^2 + \frac{1}{4}\left(\frac{\partial u}{\partial y} +\frac{\partial v}{\partial x}\right)^2 + \frac{\partial u}{\partial x}\frac{\partial v}{\partial y}\right)^{\frac{1-n}{2n}},
\end{equation}
where $n = 3$ is the flow-law exponent, and $B$ is the pre-factor dependent on ice temperature, among other factors. 


The above PDEs is what `dJUICE` is going to solve. We will only need to set the pre-factor $B$ and exponent $n$ in this tutorial.

In [ ]:
md.materials.rheology_B=1.815730284801701e+08*ones(md.mesh.numberofvertices)
md.materials.rheology_n=3*ones(md.mesh.numberofelements);

The basal shear stress is related to the ice velocity by a friciton law, here we use the Budd friction law
$$\tau_b=C^2N^\frac{q}{p}|u_b|^{\frac{1}{q}-1}u_b$$
where the effective pressure $N$ is calculated by `dJUICE` at the base of the ice.

To start, we set the basal friction coefficient $C=20~\text{m}^{-1/2} \text{s}^{1/2}$, and the exponents $p=1$ and $q=1$.


In [ ]:
md.friction.coefficient=20*ones(md.mesh.numberofvertices)
md.friction.p=ones(md.mesh.numberofvertices)
md.friction.q=ones(md.mesh.numberofvertices);

## Boundary conditions

Let's set the left, bottom, right boundaries to be Dirichlet boundary, and the top bondary to be a calving front boundary. In `dJUICE`, the calving front boundary is automatically determined by the 0-levelset contour of `md.mask.ice_levelset`, which are the nodes inside the polygon defined in `./test/Exp/SquareFront.exp`.

The Dirichelt boundaries are the rest boundaries of the domain, by setting the values in `md.stressbalance.spcvx` and `md.stressbalance.spcvy`.

In [ ]:
#Boundary conditions
nodefront=ContourToNodes(md.mesh.x,md.mesh.y,issmdir()*"/test/Exp/SquareFront.exp",2.0) .& md.mesh.vertexonboundary
md.stressbalance.spcvx = NaN*ones(md.mesh.numberofvertices)
md.stressbalance.spcvy = NaN*ones(md.mesh.numberofvertices)
pos = findall(md.mesh.vertexonboundary .& .~nodefront)
md.mask.ice_levelset[findall(nodefront)] .= 0

segmentsfront=md.mask.ice_levelset[md.mesh.segments[:,1:2]]==0
segments = findall(vec(sum(Int64.(md.mask.ice_levelset[md.mesh.segments[:,1:2]].==0), dims=2)) .!=2)
pos=md.mesh.segments[segments,1:2]
md.stressbalance.spcvx[pos] .= 0.0
md.stressbalance.spcvy[pos] .= 0.0;

## Numerical tolerance

- `restol` is the mechanical equilibrium residual convergence criterion
- `reltol` is the velocity relative convergence criterion
- `abstol` is the velocity absolute convergence criterion

If the tolerance is set to `NaN`, that means it is not applied.

In [ ]:
md.stressbalance.restol=0.05
md.stressbalance.reltol=0.05
md.stressbalance.abstol=NaN;

## Solve

Now let's solve the nonlinar PDEs

In [ ]:
md=solve(md,:Stressbalance)

## plot solutions

The solutions are in `md.results["StressbalanceSolution"]`. We can plot the velocity magnitude by the following command.

In [ ]:
plotmodel(md, md.results["StressbalanceSolution"]["Vel"]*md.constants.yts)